In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("distilled_models", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from utils.config import Config
from utils.parse_dataset import NetworkDataset, load_datasets
from utils.load_models import MLP_Models
from utils.knowledge_distillation import KnowledgeDistillation

In [2]:
conf = Config()
load_models = MLP_Models()
load_models2 = MLP_Models()
teacher_model = load_models.mlp_4
teacher_model.load()
student_model = load_models.light_mlp_1
ref_student_model = load_models2.light_mlp_1
kd = KnowledgeDistillation(
    teacher=teacher_model,
    student=student_model,
    device=conf.device,
    distillation="relation"
)

Checkpoint loaded from /global/D1/homes/jorgetf/Network-Packet-ML-Model/large_models/checkpoint/mlp_4.pth!


In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "mlp")

# create train, val and test dataloaders
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, conf.batch_size, shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, conf.batch_size, shuffle=True)

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, conf.batch_size)

In [4]:
data, labels = next(iter(train_loader))
print(data.shape, labels.shape)

torch.Size([512, 513]) torch.Size([512])


In [5]:
# train ref student model
#acc, train_loss, val_loss = ref_student_model.train(train_loader, val_loader, conf.epochs)

In [6]:
# test ref student accuracy
#loss, accuracy = ref_student_model.evaluate(test_loader)
#print(f"Student accuracy: {100*accuracy:.2f}%")

In [7]:
acc, train_loss, val_loss = kd.train_kd(train_loader, val_loader, conf.epochs)

Epoch: 1/10, Accuracy: 66.43%, Train loss: 0.0107, Val loss: 1.4588
Epoch: 2/10, Accuracy: 72.81%, Train loss: 0.0055, Val loss: 0.6822
Epoch: 3/10, Accuracy: 79.19%, Train loss: 0.0039, Val loss: 0.4684
Epoch: 4/10, Accuracy: 80.73%, Train loss: 0.0032, Val loss: 0.4271
Epoch: 5/10, Accuracy: 80.50%, Train loss: 0.0028, Val loss: 0.3688
Epoch: 6/10, Accuracy: 82.71%, Train loss: 0.0026, Val loss: 0.4110
Epoch: 7/10, Accuracy: 83.75%, Train loss: 0.0024, Val loss: 0.3089
Epoch: 8/10, Accuracy: 84.25%, Train loss: 0.0023, Val loss: 0.3834
Epoch: 9/10, Accuracy: 84.64%, Train loss: 0.0022, Val loss: 0.3440
Epoch: 10/10, Accuracy: 84.41%, Train loss: 0.0022, Val loss: 0.3303


In [8]:
# test teacher accuracy
loss, accuracy = teacher_model.evaluate(test_loader)
print(f"Teacher accuracy: {100*accuracy:.2f}%")

Teacher accuracy: 92.46%


In [9]:
# test student accuracy
loss, accuracy = student_model.evaluate(test_loader)
print(f"Student accuracy: {100*accuracy:.2f}%")

Student accuracy: 84.14%
